In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set()

In [18]:
#uploading players' stats
data = pd.read_csv("player_stats.csv")
data.drop("Unnamed: 0",axis=1,inplace=True)

In [116]:
#uploadings ballon d'or rankings
ballonsdor = {}
for year in range(1996,2020):
    ballonsdor[year] = pd.read_csv("bo_"+str(year)+".csv")

In [117]:
#forcing int type for points column
for year in tqdm(range(1996,2020)):
   try:
    ballonsdor[year]["Points"] = ballonsdor[year]["Points"].astype(int)
   except KeyError:
     print("Points column not found in "+str(year)+", proceeding")

100%|██████████| 24/24 [00:00<00:00, 2393.44it/s]

Points column not found in 2010, proceeding
Points column not found in 2011, proceeding
Points column not found in 2012, proceeding
Points column not found in 2013, proceeding
Points column not found in 2014, proceeding


In [118]:
#normalizing points won for each year
for df in tqdm(ballonsdor.keys()):
  try:
    ballonsdor[df]["%"] = ballonsdor[df]["Points"]/ballonsdor[df]["Points"].sum()
  except KeyError:
    print("Points column not found in "+str(df)+", proceeding")
  except TypeError:
    continue

100%|██████████| 24/24 [00:00<00:00, 2376.60it/s]

Points column not found in 2010, proceeding
Points column not found in 2011, proceeding
Points column not found in 2012, proceeding
Points column not found in 2013, proceeding
Points column not found in 2014, proceeding


In [119]:
#creating join key for player's stats
data["BO_JK"] = data["Season_End_Year"].astype(str) + data["Player"]

In [120]:
#creating join keys for BO rankings
for year in tqdm(ballonsdor):
    try:
        ballonsdor[year]["BO_JK"] = str(year) + ballonsdor[year]["Nom"]
    except KeyError:
        ballonsdor[year]["BO_JK"] = str(year) + ballonsdor[year]["Joueur"]

100%|██████████| 24/24 [00:00<00:00, 1543.28it/s]


In [121]:
#normalizing column names
for year in ballonsdor:
    try:    
        ballonsdor[year]["Nom"] = ballonsdor[year]["Joueur"]
        ballonsdor[year].drop("Joueur",axis=1,inplace=True)
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["%"] = ballonsdor[year]["Pourcentage"]
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Équipe nationale"]
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["%"] = ballonsdor[year]['Pourcentage[6]']
    except KeyError:
        continue
for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Sélection nationale"]
    except KeyError:
        continue
for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Nationalité"]
    except KeyError:
        continue


In [123]:
#append BO rankings together
BO_rankings = ballonsdor[1996].head()
BO_rankings.drop(0,inplace=True)
for year in ballonsdor:
    BO_rankings = BO_rankings.append(ballonsdor[year])

C:\Users\charl\miniconda3\envs\BallondOr\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [124]:
BO_rankings.isna().sum()

Rang                    46
Nom                      0
Pays                     0
Club                     0
Points                 155
Pos 1                  683
Pos 2                  676
Pos 3                  665
Pos 4                  663
Pos 5                  652
%                       13
BO_JK                    0
Sélection nationale    263
Nationalité            541
Rang[6]                686
Pourcentage[6]         686
Pourcentage            590
Unnamed: 0             686
Équipe nationale       686
Poste                  649
dtype: int64

In [125]:
#drop useless/duplicate columns
BO_rankings = BO_rankings[["BO_JK","Nom","Pays","Club","%"]]

In [126]:
BO_rankings

,BO_JK,Nom,Pays,Club,%
1,1996Ronaldo,Ronaldo,Brésil,PSV Eindhoven / FC Barcelone,0.186928
2,1996Alan Shearer,Alan Shearer,Angleterre,Blackburn Rovers / Newcastle United,0.139869
3,1996Alessandro Del Piero,Alessandro Del Piero,Italie,Juventus,0.090196
4,1996Jürgen Klinsmann,Jürgen Klinsmann,Allemagne,Bayern Munich,0.078431
0,1996Matthias Sammer,Matthias Sammer,Allemagne,Borussia Dortmund,0.188235
...,...,...,...,...,...
25,2019Karim Benzema,Karim Benzema,France,Real Madrid,0.000355
26,2019Georginio Wijnaldum,Georginio Wijnaldum,Pays-Bas,Liverpool FC,0.000355
27,2019João Félix,João Félix,Portugal,Benfica Lisbonne Atlético de Madrid,0.0
28,2019Marquinhos,Marquinhos,Brésil,Paris Saint-Germain,0.0


In [127]:
#joins
for year in tqdm(ballonsdor):
    new_data = data.merge(right=BO_rankings,on="BO_JK",how="right")

100%|██████████| 24/24 [00:00<00:00, 38.39it/s]


In [135]:
new_data.dropna(subset=["Season_End_Year"],inplace=True)

1) créer clés squad_JK et country_JK dans new_data
2) pour chaque .csv de cup winners, créer clés cup_JK_1 (winner) et cup_JK_2 (finalist)
3) pour chaque cup de club, créer une colonne qui vaut 1 si la clé est trouvée, 0 sinon
4) idem pour cup nationales

In [186]:
#1) créer clés squad_JK et Nation_JK dans new_data
new_data["Squad_JK"] = new_data["Season_End_Year"].astype(str) + new_data["Squad"]
new_data["Nation_JK"] = new_data["Season_End_Year"].astype(str) + new_data["Nation"]

In [183]:
competitions_winners = {}

In [220]:
competitions_winners["Bundesliga"] = pd.read_csv("Bundesliga_winners.csv")
competitions_winners["C3"] = pd.read_csv("C3_Winners.csv")
competitions_winners["UCL"] = pd.read_csv("Champions_League_winners.csv")
competitions_winners["Club WC"] = pd.read_csv("club_world_cup_winners.csv")
competitions_winners["Copa America"] = pd.read_csv("copa_america_winners.csv")
competitions_winners["Euro"] = pd.read_csv("euro_winners.csv")
competitions_winners["Liga"] = pd.read_csv("Liga_winners.csv")
competitions_winners["Ligue 1"] = pd.read_csv("ligue1_winners.csv")
competitions_winners['PL'] = pd.read_csv("PL_winners.csv")
competitions_winners['Serie A'] = pd.read_csv("SerieA_winners.csv")
competitions_winners['WC'] = pd.read_csv('WC_winners.csv')

In [224]:
#2) pour chaque .csv de cup winners, créer clés cup_JK_1 (winner) et cup_JK_2 (finalist)
for competition in tqdm(competitions_winners.items()):
    competitions_winners[competition[0]][competition[0]+"_JK_1"] = competitions_winners[competition[0]]["Year"].astype(str) + competitions_winners[competition[0]]["Winner"]
    try:
        competitions_winners[competition[0]][competition[0]+"_JK_2"] = competitions_winners[competition[0]]["Year"].astype(str) + competitions_winners[competition[0]]["Finalist"]
    except KeyError:
        continue

100%|██████████| 11/11 [00:00<00:00, 1338.94it/s]


In [238]:
#3) pour chaque cup de club, créer une colonne dans new_data qui vaut 1 si la clé est trouvée, 0 sinon
for competition in tqdm(competitions_winners.items()):
    new_data["winner_"+competition[0]] = new_data["Squad_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_1'].unique() else 0)
    try:
        new_data["finalist_"+competition[0]] = new_data["Squad_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_2'].unique() else 0)
    except KeyError:
        continue

#4) idem pour cup nationales
for competition in tqdm(competitions_winners.items()):
    new_data["winner_"+competition[0]] = new_data["Nation_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_1'].unique() else 0)
    try:
        new_data["finalist_"+competition[0]] = new_data["Nation_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_2'].unique() else 0)
    except KeyError:
        continue

100%|██████████| 11/11 [00:00<00:00, 25.92it/s]


In [241]:
new_data

,Season_End_Year,Squad,Comp,Player,Nation,Pos,Age,Born,MP_Playing,Starts_Playing,...,winner_Copa America,finalist_Copa America,winner_Euro,finalist_Euro,winner_Liga,winner_Ligue 1,winner_PL,winner_Serie A,winner_WC,finalist_WC
1,1996,Blackburn,Premier League,Alan Shearer,ENG,FW,24.0,1970.0,35.0,35.0,...,0,0,0,0,0,0,0,0,0,0
2,1996,Juventus,Serie A,Alessandro Del Piero,ITA,"FW,MF",20.0,1974.0,29.0,25.0,...,0,0,0,0,0,0,0,0,0,0
3,1996,Bayern Munich,Bundesliga,Jürgen Klinsmann,GER,FW,31.0,1964.0,32.0,32.0,...,0,0,1,0,0,0,0,0,0,0
4,1996,Dortmund,Bundesliga,Matthias Sammer,GER,"DF,MF",27.0,1967.0,22.0,22.0,...,0,0,1,0,0,0,0,0,0,0
6,1996,Blackburn,Premier League,Alan Shearer,ENG,FW,24.0,1970.0,35.0,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,2019,Napoli,Serie A,Kalidou Koulibaly,SEN,DF,27.0,1991.0,35.0,35.0,...,0,0,0,0,0,0,0,0,0,0
714,2019,Barcelona,La Liga,Marc-André ter Stegen,GER,GK,26.0,1992.0,35.0,35.0,...,0,0,0,0,0,0,0,0,0,0
715,2019,Real Madrid,La Liga,Karim Benzema,FRA,FW,30.0,1987.0,36.0,35.0,...,0,0,0,0,0,0,0,0,0,0
716,2019,Liverpool,Premier League,Georginio Wijnaldum,NED,MF,27.0,1990.0,35.0,32.0,...,0,0,0,0,0,0,0,0,0,0


In [181]:
test["won_UCL"] = test["Squad_JK"].apply(lambda x : 1 if x in UCL_winners['UCL_JK_1'].unique() else 0)

In [182]:
test["won_UCL"]

1      0
2      1
3      0
4      0
6      0
      ..
713    0
714    0
715    0
716    1
718    0
Name: won_UCL, Length: 619, dtype: int64

In [180]:
UCL_winners["UCL_JK_1"]

'1995Ajax Amsterdam'

In [166]:
test["won_UCL"].unique()

array([0], dtype=int64)